In [1]:
from datetime import date
import itertools
import logging
import pandas as pd
import numpy as np
import datetime
import glob
import os
import pickle
import ete3
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
logging.getLogger().setLevel(logging.INFO)
# logging.getLogger().setFormat('[%(levelname)s] %(message)s')

def setup_logging(verbose=False):
    try: 
        del logging.root.handlers[:]
    except:
        pass
    if verbose:
        logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
    else:
        logging.basicConfig(level=logging.WARNING, format='[%(levelname)s] %(message)s')
setup_logging(verbose=True)   
logging.info('test')

[INFO] test


In [3]:
fn = 'data/NE BIOSCAN_Manifest_V1.0_Yarner_2021.xlsx'
template_fn = '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'

In [4]:
# download and install taxonomy
ncbi = ete3.NCBITaxa()
# only run update if needed
# ncbi.update_taxonomy_database()

In [5]:
def get_data(fn):

    logging.info('reading data from {!r}'.format(fn))
    
    try:
        df = pd.read_excel(fn, dtype=str, index_col=0, keep_default_na=False,
                           sheet_name='TAB1 Specimen Metadata Entry')
    except:
        df = pd.read_excel(fn, dtype=str, index_col=0, keep_default_na=False,
                           sheet_name='Specimen Metadata Entry')
    
    if df.index.duplicated().any():
        logging.error('duplicate SERIES: {}'.format(df.index[df.index.duplicated()].to_list()))
        
    # trailing spaces
    for col in df.columns:
        trailing_spaces = (df[col].str.startswith(' ') | df[col].str.endswith(' '))
        if trailing_spaces.any():
            logging.warning('trailing spaces found in column {!r}, rows {}. Removing for validation'.format(col,
                df.loc[trailing_spaces].index.to_list()))
            df[col] = df[col].str.strip()
        
    return df
df = get_data(fn)

[INFO] reading data from 'data/NE BIOSCAN_Manifest_V1.0_Yarner_2021.xlsx'
[WARNING] trailing spaces found in column 'WHAT_3_WORDS', rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]. Removing for validation
[WARNING] trailing spaces fou

In [6]:
template_df = get_data(template_fn)

[INFO] reading data from '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'


In [7]:
def check_columns(df, template_df):
    
    logging.info('checking manifest columns against template')
    
    data_cols = set(df.columns)
    template_cols = set(template_df.columns)
        
    if data_cols - template_cols != set():
        logging.warning('extra columns in filled manifest compared to template: {}'.format(data_cols - template_cols))
    if template_cols - data_cols != set():
        logging.error('template columns missing from filled manifest: {}'.format(template_cols - data_cols))
check_columns(df, template_df)

[INFO] checking manifest columns against template


In [8]:
## TODO - which columns require NA, do not remove blanks to be able to get taxids for all
def check_blanks(df):
    
    logging.info('Checking and excluding blank samples')
    
    # last well of plate expected to be blank
    last_well = df[df['TUBE_OR_WELL_ID'] == 'H12']
    last_well_blanks = (last_well['SCIENTIFIC_NAME'] == 'blank sample')
    if not last_well_blanks.all():
        logging.error('last well H12 is not blank at SERIES {}: in SCIENTIFIC_NAME, '
                      'expected "blank sample", found {}'.format(
                        last_well[~last_well_blanks].index.to_list(),
                        last_well[~last_well_blanks].SCIENTIFIC_NAME.to_list()
        ))
    
    is_blank = (df['SCIENTIFIC_NAME'] == 'blank sample')
    blank_df = df[is_blank]
    
    logging.info('found {} blank samples based on SCIENTIFIC_NAME'.format(blank_df.shape[0]))
    
    # check organism part
    organism_part_pass = (blank_df['ORGANISM_PART'] == 'BLANK_SAMPLE')
    if not organism_part_pass.all():
        logging.error('for blanks, ORGANISM_PART expected to be BLANK_SAMPLE, found {}'.format(
                set(blank_df.loc[~organism_part_pass, 'ORGANISM_PART'])))
    
    # check that NOT_APPLICABLE is filled in all applicable "orange" columns
    blanks_na = blank_df[[
        'CATCH_LOT','BOTTLE_DIRECTION','HAZARD_GROUP',
        'REGULATORY_COMPLIANCE','DATE_OF_COLLECTION','COLLECTION_LOCATION',
        'DECIMAL_LATITUDE','DECIMAL_LONGITUDE','WHAT_3_WORDS' # ORDER FAMILY GENUS
    ]]
    na_filled = (blanks_na == 'NOT_APPLICABLE').all(axis=0)
    if not na_filled.all():
        logging.warning('for blanks, NOT_APPLICABLE expected, but not found in columns {}'.format(
                            na_filled[~na_filled].index.to_list()))
    # exclude blanks from downstream analysis    
    # logging.info('{} samples of {} left for downstream analysis'.format(df_flt.shape[0], df.shape[0]))
    
    return is_blank
        
blanks_mask = check_blanks(df)

[INFO] Checking and excluding blank samples
[ERROR] last well H12 is not blank at SERIES [96]: in SCIENTIFIC_NAME, expected "blank sample", found ['NOT_APPLICABLE']
[INFO] found 10 blank samples based on SCIENTIFIC_NAME


In [9]:
def get_valid_dict(fn):
    
    # pick up validation values from data validation sheet
    logging.info('extracting value validation data from {!r}'.format(fn))
    valid_df = pd.read_excel(fn, dtype=str, sheet_name='Data Validation - do not edit')
    valid_dict = dict()
    for col in valid_df.columns:
        valid_dict[col] = valid_df[col].dropna().to_list()
    
    return valid_dict
valid_dict = get_valid_dict(template_fn)

[INFO] extracting value validation data from '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'


In [10]:
def exclude_missing(series, na_values=None):
    
    # valid missing data 
    no_data = (series.isin(na_values))
    if no_data.sum() > 0:
        logging.info('excluding {} {!r} samples without data in {!r}'.format(no_data.sum(), na_values, series.name))
    return series[~no_data]
    
exclude_missing(df['TIME_OF_COLLECTION'], na_values=['NOT_COLLECTED',''])

[INFO] excluding 290 ['NOT_COLLECTED', ''] samples without data in 'TIME_OF_COLLECTION'


SERIES
example-small               11:22:00
example-large               11:22:00
example-handcaught          13:00:00
1                           11:00:00
2                           11:00:00
                           ...      
1002                        10:30:00
1003                        10:30:00
1004                        10:30:00
1005                        10:30:00
1056                  NOT_APPLICABLE
Name: TIME_OF_COLLECTION, Length: 1009, dtype: object

In [11]:
df.shape

(1299, 38)

In [12]:
def validate_series(df):
    
    # series should be 1,2, ..., nsamples
    logging.info('validating SERIES')
    
    expected_series = set([str(i) for i in range(1, df.shape[0] + 1)])
    
    observed_series = set(df.index.astype(str))
    
    if expected_series != observed_series:
        logging.error(f'In SERIES, {expected_series - observed_series} are missing, '
                      f'{observed_series - expected_series} are unexpected')
    
    
validate_series(df)

[INFO] validating SERIES
[ERROR] In SERIES, {'1299', '1297', '1298'} are missing, {'example-large', 'example-small', 'example-handcaught'} are unexpected


In [13]:
df.RACK_OR_PLATE_ID.duplicated()

SERIES
example-small         False
example-large          True
example-handcaught     True
1                     False
2                      True
                      ...  
1292                   True
1293                   True
1294                   True
1295                   True
1296                   True
Name: RACK_OR_PLATE_ID, Length: 1299, dtype: bool

In [14]:
def validate_plates_wells(df, plate_col, well_col):
    
    # expect only complete 96-well plates
    logging.info(f'validating {plate_col} and {well_col}')
    
    logging.info(f'found {df.shape[0]} samples across {df[plate_col].nunique()} plates')
    
    # add 96-well plate well IDs to validation
    row_id = list('ABCDEFGH')
    col_id = range(1,13)
    expected_wells = set([r + str(c) for (r,c) in itertools.product(row_id, col_id)])
    
    for plate, pdf in df.groupby(plate_col):
        dup_wells =  pdf[well_col].duplicated()
        if dup_wells.any():
            logging.error(f'duplicate {well_col} for plate {plate}: {pdf.loc[dup_wells, well_col].unique()}')
        observed_wells = set(pdf[well_col])
        if observed_wells != expected_wells:
            logging.error(f'in {well_col} for plate {plate}, wells {expected_wells - observed_wells} '
                          f'are missing, wells {observed_wells - expected_wells} are excessive')
        
validate_plates_wells(df, 'RACK_OR_PLATE_ID', 'TUBE_OR_WELL_ID')

[INFO] validating RACK_OR_PLATE_ID and TUBE_OR_WELL_ID
[INFO] found 1299 samples across 13 plates
[ERROR] duplicate TUBE_OR_WELL_ID for plate : ['']
[ERROR] in TUBE_OR_WELL_ID for plate , wells {'E4', 'B5', 'G8', 'E3', 'H2', 'E6', 'B7', 'H11', 'E11', 'A5', 'F3', 'D6', 'G10', 'F11', 'B4', 'C3', 'G4', 'A12', 'B12', 'F4', 'D12', 'C11', 'G7', 'H6', 'H3', 'B8', 'H5', 'C6', 'F7', 'F5', 'H1', 'G9', 'E1', 'C1', 'G5', 'A1', 'E9', 'B6', 'A8', 'B1', 'F8', 'H8', 'H10', 'H7', 'F9', 'G3', 'D10', 'A4', 'F12', 'C7', 'A7', 'C8', 'G2', 'C12', 'C2', 'A10', 'D5', 'D1', 'E7', 'D3', 'H9', 'B10', 'A6', 'D7', 'F1', 'F6', 'C4', 'D11', 'G6', 'E10', 'D8', 'G11', 'G12', 'E12', 'B2', 'B9', 'A9', 'D2', 'H12', 'F2', 'C10', 'D4', 'A2', 'A11', 'E5', 'G1', 'H4', 'C9', 'F10', 'A3', 'E2', 'C5', 'E8', 'D9', 'B3', 'B11'} are missing, wells {''} are excessive
[ERROR] in TUBE_OR_WELL_ID for plate SHAP-001, wells {'E4', 'B5', 'G8', 'E3', 'H2', 'E6', 'B7', 'H11', 'E11', 'A5', 'F3', 'D6', 'G10', 'F11', 'B4', 'C3', 'G4', 'A12', 

In [15]:
def validate_values(col, df, valid_dict, sep=None, na_values=None, level='e'):
    
    logging.info('validating values in column {!r}'.format(col))
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    if col not in valid_dict.keys():
        logging.error('{!r} column not found in validation sheet'.format(col))
        return
    assert level in ('i','w','e'), '{!r} invalid logging level for validate_values'.format(level)
    
    series = df[col]
    if na_values:
        series = exclude_missing(series, na_values)
    
    col_values = set(series.unique())
    # use separator to split values
    if sep:
        sep_col_values = list()
        for v in col_values:
            sep_col_values.extend([x.strip() for x in v.split(sep)])
        col_values = set(sep_col_values)
    valid_values = set(valid_dict[col])
    invalid_values = col_values - valid_values
    if len(invalid_values) > 0:
        msg = 'invalid values in {!r}: {}'.format(col, invalid_values)
        if level == 'i':
            logging.info(msg)
        elif level == 'w':
            logging.warning(msg)
        elif level == 'e':
            logging.error(msg)
#     else:
#         logging.info('all values valid in {!r}'.format(col))
            
validate_values('ORGANISM_PART', df, valid_dict, sep=" | ")

[INFO] validating values in column 'ORGANISM_PART'
[ERROR] invalid values in 'ORGANISM_PART': {''}


In [16]:
def validate_date(col, df, na_values=['NOT_COLLECTED','NOT_APPLICABLE']):
    
    logging.info('validating date column {!r}'.format(col))

    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    series = exclude_missing(series, na_values)
    
    # invalid date formats
    # empty string converted to NaT
    date_series = pd.to_datetime(series, format='%Y-%m-%d', errors='coerce')
    if date_series.isna().any():
        logging.error('invalid dates in {!r}: {}'.format(col, 
                                                         series[date_series.isna()].unique()))
    valid_date_series = date_series[~date_series.isna()]
    
    # dates in future
    future_dates = (valid_date_series > datetime.datetime.today())
    if future_dates.any():
        logging.error('future dates in {!r}: {}'.format(col,
            valid_date_series[future_dates].to_list()))
        
    # dates too old
    old_dates = (valid_date_series < datetime.datetime.strptime('1900-01-01', '%Y-%m-%d'))
    if old_dates.any():
        logging.error("pre-1900 dates in {!r}: {}".format(col,
            valid_date_series[old_dates].to_list())) 
    
    return valid_date_series
df.loc[1,'DATE_OF_COLLECTION'] = 'NOT_COLLECTED'
validate_date('DATE_OF_COLLECTION', df)

[INFO] validating date column 'DATE_OF_COLLECTION'
[INFO] excluding 13 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'DATE_OF_COLLECTION'
[ERROR] invalid dates in 'DATE_OF_COLLECTION': ['']


SERIES
example-small        2021-01-12
example-large        2021-01-12
example-handcaught   2021-05-14
2                    2021-06-26
3                    2021-06-26
                        ...    
1001                 2021-12-05
1002                 2021-12-05
1003                 2021-12-05
1004                 2021-12-05
1005                 2021-12-05
Name: DATE_OF_COLLECTION, Length: 996, dtype: datetime64[ns]

In [17]:
def validate_time(col, df, na_values=['NOT_COLLECTED','NOT_APPLICABLE']):
    
    logging.info('validating time column {!r}'.format(col))
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    series = exclude_missing(series, na_values)
        
    # invalid time formats
    # NB empty string converted to NaT
    time_series = pd.to_datetime(series, format='%H:%M:%S', errors='coerce')
    if time_series.isna().any():
        logging.error('invalid times in {!r}: {}'.format(col, 
                                                         series[time_series.isna()].unique()))
    valid_time_series = time_series[~time_series.isna()]
    
    return valid_time_series
# df.loc[1,'TIME_OF_COLLECTION'] = '23'
validate_time('TIME_OF_COLLECTION', df)

[INFO] validating time column 'TIME_OF_COLLECTION'
[INFO] excluding 12 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'TIME_OF_COLLECTION'
[ERROR] invalid times in 'TIME_OF_COLLECTION': ['']


SERIES
example-small        1900-01-01 11:22:00
example-large        1900-01-01 11:22:00
example-handcaught   1900-01-01 13:00:00
1                    1900-01-01 11:00:00
2                    1900-01-01 11:00:00
                             ...        
1001                 1900-01-01 10:30:00
1002                 1900-01-01 10:30:00
1003                 1900-01-01 10:30:00
1004                 1900-01-01 10:30:00
1005                 1900-01-01 10:30:00
Name: TIME_OF_COLLECTION, Length: 997, dtype: datetime64[ns]

In [18]:
def validate_time_period(col, df, na_values=['NOT_COLLECTED','NOT_APPLICABLE']):
    
    logging.info('validating time period column {!r}'.format(col))
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    series = exclude_missing(series, na_values)

    # conversion with modifications for proper parsing 
    # by pd.Timedelta (does not accept missing data, e.g. 'PT1H')
    # note - will not work for weeks and months
    def convert_iso_duration(s):
        if s == np.nan:
            return np.nan
        if not s.startswith('P') or 'T' not in s:
            return np.nan
        # add days
        if s.startswith('PT'):
            s = s.replace('PT','P0DT')
        # add trailing minutes and seconds
        if s.endswith('H'):
            s += '0M0S'
        elif s.endswith('M'):
            s += '0S'
        try:
            return pd.Timedelta(s)
        except:
            return np.nan
    time_period_series = series.apply(convert_iso_duration)
    if time_period_series.isna().any():
        logging.error('invalid times in {!r}: {}'.format(col, 
            series[time_period_series.isna()].unique()))
    valid_time_period_series = time_period_series[~time_period_series.isna()]
    return valid_time_period_series

# df.loc[1,'DURATION_OF_COLLECTION'] = 'PVT1H'
validate_time_period('DURATION_OF_COLLECTION', df);
# df['DURATION_OF_COLLECTION']

[INFO] validating time period column 'DURATION_OF_COLLECTION'
[INFO] excluding 13 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'DURATION_OF_COLLECTION'
[ERROR] invalid times in 'DURATION_OF_COLLECTION': ['']


In [19]:
# to be replaced/supported by w3w check
def check_location(df, fn):
    
    logging.info('validating country with coordinates')
    
    loc_col, lat_col, lon_col = 'COLLECTION_LOCATION', 'DECIMAL_LATITUDE', 'DECIMAL_LONGITUDE'

    try:
        loc_df_complete = df[[loc_col, lat_col, lon_col]].copy()
    except:
        logging.error('One of {!r} {!r} {!r} columns not found in manifest'.format(loc_col, lat_col, lon_col))
        return
#     loc_df_isna = (loc_df.isin(na_values)).all(axis=1)
#     if loc_df_isna.any():
#         logging.info('removing {} {!r} samples with missing data from coordinate analysis'.format(
#                 loc_df_isna.sum(), na_values))
#     loc_df_complete = loc_df[~loc_df_isna].copy()
    
    # coordinates in geopy format
    loc_df_complete['coord'] = loc_df_complete.apply(lambda x: '{}, {}'.format(
            x[lat_col], x[lon_col]), axis=1)
    
    # get location data for coordinates
    # use local copy of web query results for re-runs
    # this 
    loc_fn = fn+'_loc.pkl'
    if os.path.isfile(loc_fn):
        locations = pickle.load(open(loc_fn, "rb"))
    else:
        # web map server - openstreetmaps
        logging.info('querying coordinates')
        locator = Nominatim(user_agent='myGeocoder')
        rgeocode = RateLimiter(locator.reverse, min_delay_seconds=1)

        locations = dict()
        for c in loc_df_complete.coord.unique():
            # pre-fill with unknown country
            locations[c] = {'address':{'country':'UNKNOWN'}}
            # check coordniate correctness
            try:
                lat, lon = c.split(', ')
                lat, lon = float(lat), float(lon)
            except:
                logging.error('problem parsing coordinates {!r}'.format(c))
                continue
            if abs(lat) > 90:
                logging.error('invalid latitude {}, should be in [-90,90]'.format(lat))
                continue
            if abs(lon) > 180:
                logging.error('invalid longitude {}, should be in [-180,180]'.format(lon))
                continue
            # web query
            location = rgeocode(c, language='en-gb')
            # rgeocode returns empty location outside of counries and in some other situations
            if location is not None:
                locations[c] = location.raw

        # save locations to file
        pickle.dump(locations, open(loc_fn, "wb"))
        
    # parse country from partner input
    loc_df_complete['partner_country'] = loc_df_complete[loc_col].apply(lambda x: x.split('|')[0].strip().upper())
    
    # extract countries from location data
    loc_countries = dict()
    for coord in locations.keys():
        coord_country = locations[coord]['address']['country'].upper()
        loc_countries[coord] = coord_country
        
        partner_countries = loc_df_complete.loc[loc_df_complete.coord == coord, 'partner_country']
        if partner_countries.nunique() > 1:
            logging.error('multiple partner countries for coordinates {!r}: {}'
                          'skipping coordinate validation'.format(
                                coord, partner_countries.unique()))
            continue
        if partner_countries.shape[0] == 0:
            logging.error('no partner location found for coordinates {!r}'.format(coord))
            continue
        partner_country = partner_countries.iloc[0]
        if coord_country == 'UNKNOWN':
            logging.warning('could not locate country for coordinates {!r}, partner country {!r}'.format(
                    coord, partner_country))
        elif partner_country != coord_country:
            logging.error('country mismatch for coordinates {!r}, partner country {!r}, '
                          'coordinate country {!r}'.format(coord, partner_country, coord_country))
    
    # countries based on coordinates
    loc_df_complete['coord_country'] = loc_df_complete['coord'].replace(loc_countries)
    country_mismatch = (loc_df_complete.coord_country != loc_df_complete.partner_country)

#     if country_mismatch.any():
#         logging.error('coordinates do not match country for SERIES: {}'.format(
#                 country_mismatch[country_mismatch].index.to_list()))
    
    # location data can be re-used, e.g. as an additional field
    return loc_df_complete
# df.loc[2,'DECIMAL_LATITUDE'] = '65'
loc_test = check_location(df, fn)
loc_test

[INFO] validating country with coordinates
[ERROR] multiple partner countries for coordinates 'NOT_APPLICABLE, NOT_APPLICABLE': ['UNITED KINGDOM' 'NOT_APPLICABLE']skipping coordinate validation
[ERROR] multiple partner countries for coordinates '50.598618, -3.7209498': ['UNITED KINGDOM' '']skipping coordinate validation
[WARNING] could not locate country for coordinates ', ', partner country ''


,COLLECTION_LOCATION,DECIMAL_LATITUDE,DECIMAL_LONGITUDE,coord,partner_country,coord_country
SERIES,,,,,,
example-small,UNITED KINGDOM | ENGLAND | SAFFRON WALDEN | VI...,52.0236,0.2389,"52.0236, 0.2389",UNITED KINGDOM,UNITED KINGDOM
example-large,UNITED KINGDOM | ENGLAND | SAFFRON WALDEN | VI...,52.0236,0.2389,"52.0236, 0.2389",UNITED KINGDOM,UNITED KINGDOM
example-handcaught,UNITED KINGDOM | ENGLAND | BICESTER | CAVERSFI...,51.917197,-1.148376,"51.917197, -1.148376",UNITED KINGDOM,UNITED KINGDOM
1,UNITED KINGDOM | ENGLAND | DEVON | EAST DARTMO...,50.592446,-3.727224,"50.592446, -3.727224",UNITED KINGDOM,UNITED KINGDOM
2,UNITED KINGDOM | ENGLAND | DEVON | EAST DARTMO...,50.592446,-3.727224,"50.592446, -3.727224",UNITED KINGDOM,UNITED KINGDOM
...,...,...,...,...,...,...
1292,,,,",",,UNKNOWN
1293,,,,",",,UNKNOWN
1294,,,,",",,UNKNOWN


In [20]:
def validate_ncbi_taxonomy(df, ncbi, na_values = ['NOT_COLLECTED','NOT_APPLICABLE']):
    
    logging.info('validating taxonomy against NCBI')
    
    tax_columns = [
        'ORDER',
        'FAMILY',
        'GENUS',
        'SCIENTIFIC_NAME'
    ]        
    
    hierarchies = df[tax_columns].drop_duplicates().copy()
    
    tax_info = dict()
    
    for tax_level in tax_columns:
        
        logging.info(f'validating {tax_level} against NCBI')
        
        if tax_level not in df.columns:
                logging.error(f'{tax_level} column not found in manifest')
                continue
            
        tax_names = list(hierarchies[tax_level].unique())
        
        for na_value in na_values:
            try:
                tax_names.remove(na_value)
            except:
                pass 
            
        for i, tax_name in enumerate(tax_names):
            if len(tax_name) == 0:
                continue
            corr_tax_name = tax_name[0].upper() + tax_name[1:].lower()
            if corr_tax_name != tax_name:
                logging.error(f'{tax_level}: unexpected case for "{tax_name}", '
                              f'changing to "{corr_tax_name}" for validation')
            tax_names[i] = corr_tax_name
        
        tax_info[tax_level] = ncbi.get_name_translator(tax_names) 
        
        unmatched_names = set(tax_names) - set(tax_info[tax_level].keys())
        logging.error(f'{tax_level}: {unmatched_names} not found in NCBI Taxonomy')
        
        expected_rank = 'species' if (tax_level == 'SCIENTIFIC_NAME') else tax_level.lower()
        
        for tname, tids in tax_info[tax_level].items():
            
            ranks = ncbi.get_rank(tids)
            
            upd_tid = tids[0]
            
            if len(tids) == 1:
                if ranks[upd_tid] != expected_rank: 
                    logging.warning(f'{tax_level}: found unexpected rank for {tname} (taxid {upd_tid}): {ranks[upd_tid]}')
            if len(tids) > 1:            
                for tid, r in ranks.items():
                    if r == expected_rank and len(tids) > 1:
                        logging.info(f'{tax_level}: using only first matching rank for {tname} (taxid {tid}): {r}')
                        upd_tid = tid
                        break
                else:
                    logging.warning(f'{tax_level}: could not find matching rank for {tname}, '
                                    f'using (taxid {upd_tid}): {ranks[upd_tid]}')
                    
            tax_info[tax_level][tname] = upd_tid
        
        logging.info(f'{tax_level} {tax_info[tax_level]}')
                    
    # check correctness of taxonomy
    for _, r in hierarchies.iterrows():
        
        if r.ORDER in na_values:
            continue
        try:
            order_id = tax_info['ORDER'][r.ORDER]
        except KeyError:
            logging.warning(f'cannot validate ORDER for "{r.ORDER}", skipping taxonomy consistency check')
            continue
            
        if r.FAMILY in na_values:
            continue
        try:
            family_id = tax_info['FAMILY'][r.FAMILY]
            
            family_lineage = ncbi.get_lineage(family_id)
            
            if order_id not in family_lineage:
                logging.error(f'Family {r.FAMILY} (taxid {family_id}) does not belong to {r.ORDER} (taxid {order_id})')
        except KeyError:
            logging.warning(f'cannot validate FAMILY for "{r.FAMILY}", skipping taxonomy consistency check')
            continue
            
        if r.GENUS in na_values:
            continue
        try:
            genus_id = tax_info['GENUS'][r.GENUS]
            
            genus_lineage = ncbi.get_lineage(genus_id)
            
            if order_id not in genus_lineage:
                logging.error(f'Genus {r.GENUS} (taxid {genus_id}) does not belong to {r.ORDER} (taxid {order_id})')
            if family_id not in genus_lineage:
                logging.error(f'Genus {r.GENUS} (taxid {genus_id}) does not belong to {r.FAMILY} (taxid {family_id})')
        except KeyError:
            logging.warning(f'cannot validate GENUS for "{r.GENUS}", skipping taxonomy consistency check')
            continue
            
        if r.SCIENTIFIC_NAME in na_values:
            continue
        try:
            species_id = tax_info['SCIENTIFIC_NAME'][r.SCIENTIFIC_NAME]
            
            species_lineage = ncbi.get_lineage(species_id)
            
            if order_id not in species_lineage:
                logging.error(f'Species {r.SCIENTIFIC_NAME} (taxid {species_id}) does not belong to {r.ORDER} (taxid {order_id})')
            if family_id not in species_lineage:
                logging.error(f'Species {r.SCIENTIFIC_NAME} (taxid {species_id}) does not belong to {r.FAMILY} (taxid {family_id})')
            if genus_id not in species_lineage:
                logging.error(f'Species {r.SCIENTIFIC_NAME} (taxid {species_id}) does not belong to {r.GENUS} (taxid {genus_id})')
        except KeyError:
            logging.warning(f'cannot validate SCIENTIFIC_NAME for "{r.SCIENTIFIC_NAME}", skipping taxonomy consistency check')
            continue
            
    return df
        
                
validate_ncbi_taxonomy(df, ncbi);

[INFO] validating taxonomy against NCBI
[INFO] validating ORDER against NCBI
[ERROR] ORDER: unexpected case for "diptera", changing to "Diptera" for validation
[ERROR] ORDER: unexpected case for "Diptera and Arachnidae", changing to "Diptera and arachnidae" for validation
[ERROR] ORDER: {'', 'Tricoptera', 'Acari (subclass)', 'Diptera and arachnidae', 'Symphyleona'} not found in NCBI Taxonomy
[WARNING] ORDER: found unexpected rank for Collembola (taxid 30001): class
[INFO] ORDER: using only first matching rank for Plecoptera (taxid 50622): order
[WARNING] ORDER: found unexpected rank for Protura (taxid 29999): class
[INFO] ORDER {'Coleoptera': 7041, 'Collembola': 30001, 'Dermaptera': 27434, 'Diptera': 7147, 'Ephemeroptera': 30073, 'Hemiptera': 7524, 'Hymenoptera': 7399, 'Lepidoptera': 7088, 'Megaloptera': 50553, 'Opiliones': 43271, 'Plecoptera': 50622, 'Protura': 29999, 'Psocoptera': 30259}
[INFO] validating FAMILY against NCBI
[ERROR] FAMILY: unexpected case for "Unkown and Acari", cha

In [21]:
def validate_int(col, df, na_values=['NOT_COLLECTED','NOT_APPLICABLE']):
    
    logging.info(f'validating int format in {col}')
    
    if col not in df.columns:
        logging.error(f'{col} column not found in manifest')
        return
    series = df[col]
    series = exclude_missing(series, na_values)
    
    for val in series.unique():
        try:
            int(val)
        except:
            logging.error(f'found non-int value in {col}: "{val}"')
validate_int('TIME_ELAPSED_FROM_COLLECTION_TO_PLATING', df)

[INFO] validating int format in TIME_ELAPSED_FROM_COLLECTION_TO_PLATING
[INFO] excluding 12 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'TIME_ELAPSED_FROM_COLLECTION_TO_PLATING'
[ERROR] found non-int value in TIME_ELAPSED_FROM_COLLECTION_TO_PLATING: ""


In [22]:
bd = validate_date('DATE_OF_COLLECTION', df)
ad = validate_date('DATE_OF_PRESERVATION', df)

[INFO] validating date column 'DATE_OF_COLLECTION'
[INFO] excluding 13 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'DATE_OF_COLLECTION'
[ERROR] invalid dates in 'DATE_OF_COLLECTION': ['']
[INFO] validating date column 'DATE_OF_PRESERVATION'
[INFO] excluding 2 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'DATE_OF_PRESERVATION'
[ERROR] invalid dates in 'DATE_OF_PRESERVATION': ['']


In [23]:
ctdf = pd.concat([bd, ad], axis=1)

In [24]:
ctdf.iloc[:, 0] > ctdf.iloc[:, 1]

example-small         False
example-large         False
example-handcaught    False
2                     False
3                     False
                      ...  
1001                  False
1002                  False
1003                  False
1004                  False
1005                  False
Length: 996, dtype: bool

In [25]:
def compare_dates(before, after):
    
    ctdf = pd.concat([before, after], axis=1)
    date_conflict = ctdf[before.name] > ctdf[after.name]
    
#     logging.info(date_conflict)
    if date_conflict.any():
        logging.error(f'{before.name} values are later than {after.name} for SERIES'
                      f' {ctdf[date_conflict].index.to_list()}')
    
compare_dates(bd, ad)

In [26]:
def validate(fn, template_fn, verbose=False, version='1.0'):
    '''
    Validation follows the order of columns order in data entry sheet
    '''

    setup_logging(verbose=verbose)

    logging.info('# started validate_partner_manifest_v.{}'.format(version))

    # read data
    df = get_data(fn)
    
    # read taxonomy
    ncbi = ete3.NCBITaxa()
    
    # prepare for validation
    template_df = get_data(template_fn)
    check_columns(df, template_df)
    valid_dict = get_valid_dict(template_fn)

    # check blanks
    blanks_mask = check_blanks(df)
    
    #orange cols
    validate_series(df)
    validate_plates_wells(df, 'RACK_OR_PLATE_ID', 'TUBE_OR_WELL_ID')
    validate_values('PRESERVATIVE_SOLUTION', df, valid_dict)
    validate_values('TUBE_OR_WELL_ID', df, valid_dict)
    # CATCH_LOT not checked TODO do not allow missing
    validate_values('BOTTLE_DIRECTION', df, valid_dict)
    validate_values('ORGANISM_PART', df, valid_dict, sep='|')
    validate_values('HAZARD_GROUP', df, valid_dict)
    validate_values('REGULATORY_COMPLIANCE', df, valid_dict)
    date_coll = validate_date('DATE_OF_COLLECTION', df, na_values=['NOT_APPLICABLE'])
    check_location(df, fn)
    
    # purple cols
    # taxonomy validation adds a few columns
    df = validate_ncbi_taxonomy(df, ncbi, na_values = ['NOT_COLLECTED', 'NOT_APPLICABLE'])
    validate_values('SEX', df, valid_dict)
    # HABITAT not checked
    validate_time('TIME_OF_COLLECTION', df)
    validate_time_period('DURATION_OF_COLLECTION', df)
    validate_values('COLLECTION_METHOD', df, valid_dict)
    # DESCRIPTION_OF_COLLECTION_METHOD not checked
    validate_int('TIME_ELAPSED_FROM_COLLECTION_TO_PLATING', df)
    # PHOTOGRAPH_* columns not checked
    # VOUCHER_ID not checked
    # PRESERVATION_APPROACH not checked - should match DATE_OF_PRESERVATION
    date_pres = validate_date('DATE_OF_PRESERVATION', df) # allow for empty values unlike DATE_OF_COLLECTION
    compare_dates(before=date_coll, after=date_pres)
    # COLLECTOR_SAMPLE_ID not checked
    validate_int('ELEVATION', df)
    # OTHER_INFORMATION	MISC_METADATA	IDENTIFIED_BY	IDENTIFIER_AFFILIATION	IDENTIFIED_HOW not checked
        
    logging.info('# ended validate_partner_manifest_v.{}'.format(version))

    return df

# fn = '../../results/partner_manifests/IRD-Neandersquito_T222Amplicon_Manifest_V2.0.xlsx'
df = validate(fn, template_fn, verbose=True)

[INFO] # started validate_partner_manifest_v.1.0
[INFO] reading data from 'data/NE BIOSCAN_Manifest_V1.0_Yarner_2021.xlsx'
[WARNING] trailing spaces found in column 'WHAT_3_WORDS', rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]. Remo

[ERROR] invalid values in 'COLLECTION_METHOD': {''}
[INFO] validating int format in TIME_ELAPSED_FROM_COLLECTION_TO_PLATING
[INFO] excluding 12 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'TIME_ELAPSED_FROM_COLLECTION_TO_PLATING'
[ERROR] found non-int value in TIME_ELAPSED_FROM_COLLECTION_TO_PLATING: ""
[INFO] validating date column 'DATE_OF_PRESERVATION'
[INFO] excluding 2 ['NOT_COLLECTED', 'NOT_APPLICABLE'] samples without data in 'DATE_OF_PRESERVATION'
[ERROR] invalid dates in 'DATE_OF_PRESERVATION': ['']
[INFO] validating int format in ELEVATION
[ERROR] found non-int value in ELEVATION: ""
[INFO] # ended validate_partner_manifest_v.1.0


In [27]:
fn = 'data/Mike Ashworth NE 2021-06-24 BIOSCAN_Manifest_V1.0.xlsx'

In [28]:
# TODO debug last well H12 is not blank at SERIES [96]; Diptera not in NCBI; unexpected rank - specify expected
df = validate(fn, template_fn, verbose=False)

[WARNING] for blanks, NOT_APPLICABLE expected, but not found in columns ['CATCH_LOT', 'BOTTLE_DIRECTION', 'WHAT_3_WORDS']
[ERROR] duplicate TUBE_OR_WELL_ID for plate : ['']
[ERROR] in TUBE_OR_WELL_ID for plate , wells {'E4', 'B5', 'G8', 'E3', 'H2', 'E6', 'B7', 'H11', 'E11', 'A5', 'F3', 'D6', 'G10', 'F11', 'B4', 'C3', 'G4', 'A12', 'B12', 'F4', 'D12', 'C11', 'G7', 'H6', 'H3', 'B8', 'H5', 'C6', 'F7', 'F5', 'H1', 'G9', 'E1', 'C1', 'G5', 'A1', 'E9', 'B6', 'A8', 'B1', 'F8', 'H8', 'H10', 'H7', 'F9', 'G3', 'D10', 'A4', 'F12', 'C7', 'A7', 'C8', 'G2', 'C12', 'C2', 'A10', 'D5', 'D1', 'E7', 'D3', 'H9', 'B10', 'A6', 'D7', 'F1', 'F6', 'C4', 'D11', 'G6', 'E10', 'D8', 'G11', 'G12', 'E12', 'B2', 'B9', 'A9', 'D2', 'H12', 'F2', 'C10', 'D4', 'A2', 'A11', 'E5', 'G1', 'H4', 'C9', 'F10', 'A3', 'E2', 'C5', 'E8', 'D9', 'B3', 'B11'} are missing, wells {''} are excessive
[ERROR] invalid values in 'PRESERVATIVE_SOLUTION': {''}
[ERROR] 'TUBE_OR_WELL_ID' column not found in validation sheet
[ERROR] invalid values i

In [30]:
locations = pickle.load(open('data/NE BIOSCAN_Manifest_V1.0_Yarner_2021.xlsx_loc.pkl', "rb"))
# locations